# Plan.

 Drop rows if needed.
 Shrink the dataset to 15-20k rows according to popularity.
 Drop columns if needed, select only those that will be interesting for clusterin model.


Quiz will ask a person to pick few favourite movies from a list of 20-50 most popular ones we have.

1) Network based
 First plan is to create a network of viewers and cluster according to movie taste. That will be done with 'ratings' dataset. Then once I have new customer movie taste I will try to find a cluster of similar customers and recommend movies he liked.

Example: there will be cluster of customers that gave high scores to comedies and if new person happen to like comedies then he will belong to this cluster and movies people in this cluster liked will be recommended.
 
2) Similar movies?
 Second one, I have movies he liked I can try to find similar movies to them and as an output get say 10 similar movies (then recommended movie picked at random say). 

Example:?

In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import ast 

from statistics import mean
#from sklearn.mixture import DPGMM

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.read_csv("movies_metadata.csv")
df_ratings = pd.read_csv("ratings_small.csv") #will work on small one 
df_keywords = pd.read_csv("keywords.csv")     #for computational reasons  
df_credits = pd.read_csv("credits.csv")

C:\Apps\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Defining few functions to be used later on.

In [3]:
def description(dataframe):
    for col in dataframe:
        print(dataframe[col].describe())

Here I am making subsets as I will be working only on 'df', Greg will be doing NLP on "df_nlp". Also I will merge later 'df_json' once I find out how to encode it.

In [4]:
df_credits['id'] = df_credits['id'].apply(str)
df = df.merge(df_credits, on = "id", how ="right")
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [5]:
df_json = df[["id", 'cast', 'crew', "belongs_to_collection", "genres"]]
df_json.to_csv(r'C:\Users\mk1u17\Downloads\df_json.csv', index = None, header=True)

In [6]:
drop_cols = ["belongs_to_collection", "genres", "production_countries",
              "spoken_languages", 'production_companies', "overview","tagline", "title", "original_title", 'cast', 'crew']

df = df.drop(drop_cols, axis=1)

I will drop few columns to narrow it down to smaller subset.

In [7]:
df_dropped = df[['homepage', 'imdb_id', 'video', 'poster_path', 'status']]
df = df.drop(['homepage', 'imdb_id', 'video', 'poster_path', 'status'], axis=1)

In [8]:
df.shape

(45538, 10)

# Cleaning metadata. 

## Missing values.

In [9]:
df.columns
df.head(5)
df.info()

Index(['adult', 'budget', 'id', 'original_language', 'popularity',
       'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count'],
      dtype='object')

,adult,budget,id,original_language,popularity,release_date,revenue,runtime,vote_average,vote_count
0,False,30000000,862,en,21.9469,1995-10-30,373554033.0,81.0,7.7,5415.0
1,False,65000000,8844,en,17.0155,1995-12-15,262797249.0,104.0,6.9,2413.0
2,False,0,15602,en,11.7129,1995-12-22,0.0,101.0,6.5,92.0
3,False,16000000,31357,en,3.85949,1995-12-22,81452156.0,127.0,6.1,34.0
4,False,0,11862,en,8.38752,1995-02-10,76578911.0,106.0,5.7,173.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45538 entries, 0 to 45537
Data columns (total 10 columns):
adult                45538 non-null object
budget               45538 non-null object
id                   45538 non-null object
original_language    45527 non-null object
popularity           45535 non-null object
release_date         45451 non-null object
revenue              45535 non-null float64
runtime              45278 non-null float64
vote_average         45535 non-null float64
vote_count           45535 non-null float64
dtypes: float64(4), object(6)
memory usage: 3.8+ MB


In [10]:
df.isnull().sum()

adult                  0
budget                 0
id                     0
original_language     11
popularity             3
release_date          87
revenue                3
runtime              260
vote_average           3
vote_count             3
dtype: int64

In [11]:
all = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/2919).sort_values(ascending=False)
missing = pd.concat([all, percent], axis= 1, keys = ["Total", "Percent"])
missing = missing.drop(missing[(missing["Total"]==0)].index)
missing #all missing values

missing.index

,Total,Percent
runtime,260,0.089072
release_date,87,0.029805
original_language,11,0.003768
vote_count,3,0.001028
vote_average,3,0.001028
revenue,3,0.001028
popularity,3,0.001028


Index(['runtime', 'release_date', 'original_language', 'vote_count',
       'vote_average', 'revenue', 'popularity'],
      dtype='object')

In [12]:
df.shape

(45538, 10)

In [13]:
df = df.dropna()

In [14]:
df.isnull().sum()
df.shape
df.columns

adult                0
budget               0
id                   0
original_language    0
popularity           0
release_date         0
revenue              0
runtime              0
vote_average         0
vote_count           0
dtype: int64

(45194, 10)

Index(['adult', 'budget', 'id', 'original_language', 'popularity',
       'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count'],
      dtype='object')

Problems:

1) Change 'adults' to binary.
2) Make sure floats columns ('budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count') are only floats, not strings. Remove if some of rows are weird.
3) Remove observations so that we get around 10-15k observations (remove according to 'popularity', we want more recognisable movies).
3) Feature engineering:
   1) If just one person gave good rating it may not be so good but 'vote_average' will still be high -> 'reliability'='vote_average'*'vote_count'
   2) 'popularity'*'reliability'
   
 Final goal is to have some way to quantify how a movie good is. 

# 'budget'

In [15]:
df['budget'] = df[~df['budget'].str.startswith('/')]['budget'].astype(int)

## 'vote_average', 'vote_count'

In [16]:
np.dtype(df['vote_average']) #alright, no strings
np.dtype(df['vote_count'])

dtype('float64')

dtype('float64')

## 'popularity'

In [17]:
df['popularity'] = df['popularity'].astype(float)
#df['popularity'].sum() #additive -> floats

In [18]:
df['popularity'].describe()

count    45194.000000
mean         2.942207
std          6.019998
min          0.000000
25%          0.398189
50%          1.139969
75%          3.730199
max        547.488298
Name: popularity, dtype: float64

In [19]:
#normalize to [1,10] 

end = 10
start = 1
arr = df['popularity']
width = end - start
res = (arr - arr.min())/(arr.max() - arr.min()) * width + start

df['popularity'] = res

In [20]:
from statistics import mean
mean(df['popularity'])

1.0483660779715207

## 'adult'

In [21]:
df['adult'].unique()

array(['False', 'True'], dtype=object)

In [22]:
df = df.replace({"adult": {'False': 0, 'True' : 1}})
df['adult'].unique()

array([0, 1], dtype=int64)

## 'original_languages' and exporting files

Select subset with English as main, drop others and then drop the whole column so that all movies are in English as original language.

In [23]:
df['original_language'] = df[df['original_language'].str.startswith('en')]['original_language']

In [24]:
df.shape
df.isna().sum()

df = df.dropna()

df.shape
df['original_language'].unique()
df.isna().sum()

(45194, 10)

adult                    0
budget                   0
id                       0
original_language    12950
popularity               0
release_date             0
revenue                  0
runtime                  0
vote_average             0
vote_count               0
dtype: int64

(32244, 10)

array(['en'], dtype=object)

adult                0
budget               0
id                   0
original_language    0
popularity           0
release_date         0
revenue              0
runtime              0
vote_average         0
vote_count           0
dtype: int64

In [25]:
df = df.drop(['original_language'], axis=1)

In [26]:
df.shape
df.head(5)
df[df['adult']==1] #only 4 movies are for adults so let's drop it
df = df.drop(['adult'], axis=1)

(32244, 9)

,adult,budget,id,popularity,release_date,revenue,runtime,vote_average,vote_count
0,0,30000000,862,1.360779,1995-10-30,373554033.0,81.0,7.7,5415.0
1,0,65000000,8844,1.279713,1995-12-15,262797249.0,104.0,6.9,2413.0
2,0,0,15602,1.192545,1995-12-22,0.0,101.0,6.5,92.0
3,0,16000000,31357,1.063445,1995-12-22,81452156.0,127.0,6.1,34.0
4,0,0,11862,1.137880,1995-02-10,76578911.0,106.0,5.7,173.0


,adult,budget,id,popularity,release_date,revenue,runtime,vote_average,vote_count
28785,1,0,169158,1.022188,1998-01-11,0.0,91.0,4.8,5.0
39979,1,0,35731,1.009662,2008-05-13,0.0,0.0,6.3,8.0
39980,1,0,55774,1.006858,2009-11-17,0.0,90.0,3.3,7.0
41085,1,750000,376004,1.018638,2016-01-05,0.0,80.0,5.2,16.0


Dataset is now clean. It basically calculates how good movie is. Now feature engineering.

In [27]:
df['reliability'] = df['vote_count']*df['vote_average']
df['greatness_scale'] = df['popularity']*df['vote_average']

In [28]:
#normalizing new features

end = 10
start = 1
arr1 = df['reliability']
arr2 = df['greatness_scale']
width = end - start
res1 = (arr1 - arr1.min())/(arr1.max() - arr1.min()) * width + start
res2 = (arr2 - arr2.min())/(arr2.max() - arr2.min()) * width + start

df['reliability'] = res1
df['greatness_scale'] = res2

df['greatness_scale'].describe()
df['reliability'].describe()

count    32244.000000
mean         1.821175
std          0.311381
min          1.000000
25%          1.709132
50%          1.870391
75%          1.997836
max         10.000000
Name: greatness_scale, dtype: float64

count    32244.000000
mean         1.074515
std          0.326492
min          1.000000
25%          1.001381
50%          1.004689
75%          1.020249
max         10.000000
Name: reliability, dtype: float64

In [29]:
df.head(3)
df.shape

,budget,id,popularity,release_date,revenue,runtime,vote_average,vote_count,reliability,greatness_scale
0,30000000,862,1.360779,1995-10-30,373554033.0,81.0,7.7,5415.0,4.291533,2.473469
1,65000000,8844,1.279713,1995-12-15,262797249.0,104.0,6.9,2413.0,2.314364,2.241722
2,0,15602,1.192545,1995-12-22,0.0,101.0,6.5,92.0,1.047207,2.090061


(32244, 10)

In [30]:
df = df[df['reliability']>= 1.004689]

In [34]:
df_dropped['release_date'] = df['release_date']
df = df.drop(['release_date'], axis=1)

In [35]:
df.shape
df.columns

(16133, 9)

Index(['budget', 'id', 'popularity', 'revenue', 'runtime', 'vote_average',
       'vote_count', 'reliability', 'greatness_scale'],
      dtype='object')

In [36]:
df = df.reset_index()

df = df.drop(['index'], axis=1)

df.head(5)
df.tail(5)

,budget,id,popularity,revenue,runtime,vote_average,vote_count,reliability,greatness_scale
0,30000000,862,1.360779,373554033.0,81.0,7.7,5415.0,4.291533,2.473469
1,65000000,8844,1.279713,262797249.0,104.0,6.9,2413.0,2.314364,2.241722
2,0,15602,1.192545,0.0,101.0,6.5,92.0,1.047207,2.090061
3,16000000,31357,1.063445,81452156.0,127.0,6.1,34.0,1.016373,1.912237
4,0,11862,1.137880,76578911.0,106.0,5.7,173.0,1.077845,1.912082


,budget,id,popularity,revenue,runtime,vote_average,vote_count,reliability,greatness_scale
16128,0,455661,1.342283,0.0,4.0,8.3,146.0,1.095662,2.566696
16129,0,39562,1.012621,0.0,89.0,5.0,12.0,1.004737,1.711999
16130,0,14008,1.072205,0.0,101.0,5.2,145.0,1.059522,1.784050
16131,0,49271,1.011919,0.0,6.0,6.7,12.0,1.006347,1.953418
16132,0,30840,1.093434,0.0,104.0,5.7,26.0,1.011699,1.876455


Playground:

def frequency(column):
    unique_list = list(column.unique())
    for string in unique_list:
        if string in ['en', 'fr', 'it', 'de', 'ja', 'es', 'hi']:
            indices = column.index(string)
            print(indices)

vowels = ['a', 'e', 'i', 'o', 'i', 'u', 'a', 'a', 'i']
indices = []
popular = ['a', 'i']
for string in vowels:
    if string in popular:
        indices.append(index(string))



def fn(df):
    listek = []
    for value in df["popularity"]:
        if type(value)==str:
            listek.append(value)
        listek = [float(x) for x in listek]
    return listek

fn(df)